In [1]:
import pandas as pd

import plotly.plotly as py
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import main
import representatives

# What if France had an Electoral College

## loading election results

In [2]:
results = main.load_results()
results.head()

,em_pct,mlp_pct,ff_pct,jlm_pct,bh_pct,nda_pct,jl_pct,pp_pct,fa_pct,na_pct,jc_pct,winner
department,,,,,,,,,,,,
Ain,22.62,25.00,21.43,15.88,5.13,6.07,1.06,0.95,1.11,0.57,0.18,mlp
Aisne,17.94,35.67,16.30,16.99,4.24,5.08,0.79,1.10,0.75,0.96,0.19,mlp
Allier,23.72,22.34,18.94,19.91,5.52,5.09,1.55,1.21,0.74,0.80,0.18,em
Alpes-de-Haute-Provence,20.02,24.53,18.49,22.51,5.00,4.87,1.73,1.18,0.93,0.52,0.21,mlp
Hautes-Alpes,21.80,21.25,19.15,21.62,5.88,5.68,1.85,1.21,0.90,0.47,0.19,em


first round results:

In [3]:
rawalloc = results.loc['Total', :]
rawalloc = rawalloc.rename(lambda nm: nm.replace('_pct', ''))
rawalloc = rawalloc[:-1]
rawalloc

em     24.01
mlp     21.3
ff     20.01
jlm    19.58
bh      6.36
nda      4.7
jl      1.21
pp      1.09
fa      0.92
na      0.64
jc      0.18
Name: Total, dtype: object

In [4]:
results = results.drop('Total')
results = results.reset_index()

## loading population statistics

In [5]:
population = main.load_population()
population.head()

,department,population
0,Nord,2595536
1,Paris,2229621
2,Bouches-du-Rhône,1993177
3,Rhône,1779845
4,Hauts-de-Seine,1591403


## adding in electoral college votes

In [6]:
population.loc[:, 'num_sen'] = 2
reps = representatives.num_reps(
    population[['department', 'population']], regionColname='department'
)
population = population.merge(reps, on='department')
population.loc[:, 'evs'] = population.num_sen + population.num_reps
population.head()

,department,population,num_sen,num_reps,evs
0,Nord,2595536,2,35.0,37.0
1,Paris,2229621,2,30.0,32.0
2,Bouches-du-Rhône,1993177,2,26.0,28.0
3,Rhône,1779845,2,24.0,26.0
4,Hauts-de-Seine,1591403,2,21.0,23.0


## assigning votes

In [7]:
tally = population.merge(results, on='department')
tally

,department,population,num_sen,num_reps,evs,em_pct,mlp_pct,ff_pct,jlm_pct,bh_pct,nda_pct,jl_pct,pp_pct,fa_pct,na_pct,jc_pct,winner
0,Nord,2595536,2,35.0,37.0,19.85,28.22,16.75,21.28,5.65,4.82,0.63,0.97,0.85,0.81,0.17,mlp
1,Paris,2229621,2,30.0,32.0,34.83,4.99,26.45,19.56,10.18,1.67,0.51,0.63,0.77,0.27,0.14,em
2,Bouches-du-Rhône,1993177,2,26.0,28.0,19.37,27.28,19.76,22.02,4.53,3.85,0.96,0.76,0.90,0.39,0.17,mlp
3,Rhône,1779845,2,24.0,26.0,26.58,16.26,23.16,19.70,6.76,4.33,0.75,0.80,1.03,0.46,0.16,em
4,Hauts-de-Seine,1591403,2,21.0,23.0,32.30,7.64,29.14,18.28,7.19,2.69,0.60,0.63,1.06,0.31,0.17,em
5,Seine-Saint-Denis,1552482,2,21.0,23.0,24.04,13.59,12.76,34.02,8.41,3.07,0.58,1.12,1.61,0.60,0.20,jlm
6,Gironde,1505517,2,20.0,22.0,26.13,18.26,17.08,21.85,7.56,4.18,1.93,1.56,0.82,0.47,0.18,em
7,Pas-de-Calais,1465205,2,19.0,21.0,18.45,34.35,14.29,19.13,5.17,4.97,0.70,1.08,0.66,1.04,0.17,mlp
8,Yvelines,1418484,2,19.0,21.0,28.86,12.92,27.25,16.65,6.93,4.34,0.71,0.72,1.07,0.38,0.18,em
9,Seine-et-Marne,1365200,2,18.0,20.0,23.11,22.85,17.77,20.84,5.70,6.10,0.76,0.93,1.20,0.54,0.18,em


electoral vote totals:

In [8]:
ectotal = tally.groupby('winner').evs.sum().sort_values(ascending=False)
ectotal

winner
em     505.0
mlp    474.0
jlm     42.0
ff      42.0
Name: evs, dtype: float64

electoral vote percentages

In [9]:
ecalloc = ectotal / ectotal.sum() * 100

## in pictures

the difference between true proportional allocation and electoral college allocation: 

In [10]:
ecalloc

winner
em     47.507056
mlp    44.590781
jlm     3.951082
ff      3.951082
Name: evs, dtype: float64

In [11]:
rawalloc

em     24.01
mlp     21.3
ff     20.01
jlm    19.58
bh      6.36
nda      4.7
jl      1.21
pp      1.09
fa      0.92
na      0.64
jc      0.18
Name: Total, dtype: object

In [12]:
compalloc = pd.concat([rawalloc, ecalloc], axis=1)
compalloc.columns = ['true', 'electoral_college']
compalloc = compalloc.fillna(0)
compalloc = compalloc.sort_values(by='true', ascending=False)
compalloc = compalloc.rename({
    'em': 'Emmanuel Macron',
    'mlp': 'Marine Le Pen',
    'ff': 'François Fillon',
    'jlm': 'Jean-Luc Mélenchon',
    'bh': 'Benoît Hamon',
    'nda': 'Nicolas Dupont-Aignan',
    'jl': 'Jean Lassalle', 
    'pp': 'Philippe Poutou',
    'fa': 'François Asselineau',
    'na': 'Nathalie Arthaud',
    'jc': 'Jacques Cheminade',
})
compalloc

,true,electoral_college
Emmanuel Macron,24.01,47.507056
Marine Le Pen,21.30,44.590781
François Fillon,20.01,3.951082
Jean-Luc Mélenchon,19.58,3.951082
Benoît Hamon,6.36,0.000000
Nicolas Dupont-Aignan,4.70,0.000000
Jean Lassalle,1.21,0.000000
Philippe Poutou,1.09,0.000000
François Asselineau,0.92,0.000000
Nathalie Arthaud,0.64,0.000000


In [13]:
x = compalloc.columns

data = [
    go.Bar(
        x=x,
        y=y,
        name=name
    )
    for (name, y) in compalloc.iterrows()
]

layout = go.Layout(barmode='stack', title='French Election Results')

fig = go.Figure(data=data, layout=layout)

iplot(fig)

The above item won't appear on github (run it in your own jupyter notebook session if you want to see it directly).

In the meantime, I've uploaded [a version of it](https://plot.ly/~r.zach.lamberty/97/) to plotly -- check it out there.